In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:

from collections import defaultdict
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm


from termcolor import colored
from IPython.display import display, Markdown
from sklearn.metrics import r2_score, classification_report, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn import preprocessing

from storage import session_scope, RTReview, MovieAward
from utils import get_movie_df
from prediction import evaluate
from constants import OSCARS_BEST_FILM

In [3]:
movie_df = get_movie_df()

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data

In [9]:
movie_df = movie_df[movie_df.year >= 2000]

In [10]:
train_df, test_df = movie_df[movie_df.year < 2018], movie_df[movie_df.year == 2018]

In [11]:
X = train_df.drop(['title', 'year', 'winner'], axis=1)
y = train_df['winner']

X_test = test_df.drop(['title', 'year', 'winner'], axis=1)

In [31]:
model = SVC(probability=True).fit(X, y)

In [57]:
model = SGDClassifier(loss='log', alpha=0.1).fit(X, y)

In [63]:
model = GradientBoostingClassifier(loss='exponential', learning_rate=0.2).fit(X, y)

In [64]:
test_df['pred_score'] = model.predict_proba(X_test)[:, 1]
test_df[['title', 'winner', 'pred_score']]

,title,winner,pred_score
358,The Shape of Water,True,3.353965e-04
359,Call Me by Your Name,False,3.218845e-06
360,Darkest Hour,False,7.460447e-11
361,Dunkirk,False,1.040694e-04
362,Get Out,False,4.916522e-07
363,Lady Bird,False,3.685593e-10
364,The Post,False,4.401198e-11
365,"Three Billboards Outside Ebbing, Missouri",False,9.001727e-06


In [54]:
df = train_df.copy()
df['pred_score'] = model.predict_proba(X)[:, 1]
df[['title', 'winner', 'pred_score']]

,title,winner,pred_score
237,American Beauty,True,0.842330
238,The Cider House Rules,False,0.061634
239,The Green Mile,False,0.082397
240,The Insider,False,0.116393
241,The Sixth Sense,False,0.021961
242,Gladiator,True,0.534471
243,Chocolat,False,0.018697
244,"Crouching Tiger, Hidden Dragon",False,0.207019
245,Erin Brockovich,False,0.053349
246,Traffic,False,0.154964
